In [5]:
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse


In [6]:
import re

In [7]:
def get_request(url):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    return response

In [8]:
def quote_scraper(response):
    quotes = response.css("div.quote > span.text::text").extract()
    author = response.css("small.author::text").extract()
    tags_div =response.css("div.tags ")
    tags = [i.css("a.tag::text").extract() for i in tags_div]
    hyper = response.css("small.author ~ a::attr(href)").extract()
    base_link = 'http://quotes.toscrape.com'
    hyperlink = [base_link + i for i in hyper]
    return pd.DataFrame({"Quotes":quotes,"Authors":author,"Tags":tags,"Hyperlinks":hyperlink})

In [9]:
pages = []
url = "http://quotes.toscrape.com/"

In [10]:
while True:
    response = get_request(url)
    pages.append(quote_scraper(response))
    next_page_url = response.css("li.next > a::attr(href)").extract_first()
    if next_page_url:
        url = response.urljoin(next_page_url)
    else:
        break

pg = pd.concat(pages)
pg

,Quotes,Authors,Tags,Hyperlinks
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]",http://quotes.toscrape.com/author/Albert-Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]",http://quotes.toscrape.com/author/J-K-Rowling
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]",http://quotes.toscrape.com/author/Albert-Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]",http://quotes.toscrape.com/author/Jane-Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]",http://quotes.toscrape.com/author/Marilyn-Monroe
...,...,...,...,...
5,“You never really understand a person until yo...,Harper Lee,[better-life-empathy],http://quotes.toscrape.com/author/Harper-Lee
6,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,...",http://quotes.toscrape.com/author/Madeleine-LE...
7,“Never tell the truth to people who are not wo...,Mark Twain,[truth],http://quotes.toscrape.com/author/Mark-Twain
8,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational],http://quotes.toscrape.com/author/Dr-Seuss
